In [ ]:
import numpy as np
import pandas as pd
import sys

libpath = './../../bgl_model/'
sys.path.append(libpath)

from main_model import *

In [ ]:
sigma_noise = 5  # noise std. dev.
bg_omin = 70
bg_omax = 180
postbg_target = 112.5

global_tmi = 0
global_tmbg = 150

# EXPERIMENT SETUP
num_event = 30  # num. of different meal events in each run per patient.
rec_per_event = 15  # num. of recommendations per meal event in each run per patient.
num_rec = num_event*rec_per_event  # num. of total recommendations

#  context variables.
cvars = ['patient', 'meal', 'bg_fasting', 'ins_salelts', 'ins_lelts', 'ins_calc_tc', \
        'SALE-LTS', 'LE-LTS', 'Tuned Calc.', 'SALE-LTS-N', 'LE-LTS-N', 'Tuned Calc.-N']

cvars_nomeal = ['patient', 'meal', 'bg_fasting', 'ins_salelts'] 

#  patient list.
all_patients = ['adult#001', 'adult#002', 'adult#003', 'adult#004', 'adult#005',
                'adult#006', 'adult#007', 'adult#008', 'adult#009', 'adult#010',
                'child#001', 'child#002', 'child#003', 'child#004', 'child#005', 
                'child#006', 'child#007', 'child#008', 'child#009', 'child#010',
                'adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 
                'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010']

child_patients = ['child#001', 'child#002', 'child#003', 'child#004', 'child#005', 'child#006', 'child#007', 'child#009', 'child#010']

normal_patients = ['adult#002', 'adult#003', 'adult#004', 'adult#006', 'adult#007', 'adult#008', \
                   'adolescent#001', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#009', 'adolescent#010']

hard_patients = ['adult#001', 'adult#005', 'adult#010', 'adolescent#007', 'child#008']

extreme_patients = ['adolescent#002', 'adolescent#008', 'adult#009']

## use these to make sure the pump always makes safe recommendations (tuned calculator)

cr_correct = {'adult#001':7, 'adult#002':1, 'adult#003':0, 'adult#004':0, 'adult#005':2,
                 'adult#006':4, 'adult#007':0, 'adult#008':3, 'adult#009':3, 'adult#010':3,
                 'adolescent#001': 0, 'adolescent#002': 3.5, 'adolescent#003': 8, 'adolescent#004': 8, 'adolescent#005': 5,
                 'adolescent#006': 0, 'adolescent#007': 6, 'adolescent#008': 2.9, 'adolescent#009': 8, 'adolescent#010': 4,
                 'child#001':-10, 'child#002':7, 'child#003':8, 'child#004':-10, 'child#005':-3,
                 'child#006':7, 'child#007':0, 'child#008':11, 'child#009':-20, 'child#010':5}

cf_correct = {'adult#001':0, 'adult#002':0, 'adult#003':0, 'adult#004':0, 'adult#005':0,
                 'adult#006':0, 'adult#007':0, 'adult#008':0, 'adult#009':0, 'adult#010':0,
                 'adolescent#001': 0, 'adolescent#002': 0, 'adolescent#003': 0, 'adolescent#004': 0, 'adolescent#005': 0,
                 'adolescent#006': 0, 'adolescent#007': 0, 'adolescent#008': 7, 'adolescent#009': 0, 'adolescent#010': 0,
                 'child#001':-30, 'child#002':0, 'child#003':0, 'child#004':-20, 'child#005':0,
                 'child#006':0, 'child#007':0, 'child#008':22, 'child#009':0, 'child#010':0}

df = pd.read_csv('./../Quest.csv')

In [ ]:
def std_bolus_calc(cho, gm, gt, cr, cf, iob=None):
    _iob = iob if iob else 0
    return np.max([0, cho / cr + (gm - gt) / cf - _iob])

In [ ]:
def create_model(data_init, ins_ceiling, postbg_target, bg_omin, bg_omax):
    variables = [{'name': 'meal', 'domain': (20, 80), 'linearbounds':(0,9), 'unit': 'g'},
            {'name': 'insulin', 'domain': (0, ins_ceiling), 'linearbounds':(-9,0), 'unit': 'U'},
            {'name': 'bg_fasting', 'domain': (50, 400), 'linearbounds':(0,9), 'unit': 'mg/dl'}]

    bgl_model = BGL_MODEL(postBG_target_range=(bg_omin, bg_omax), postBG_target_value=postbg_target, variables=variables)
    
    for ind in range(len(data_init)):
        new_x = np.array([[data_init.loc[ind,'meal']],[data_init.loc[ind,'bg_fasting']], [data_init.loc[ind,'insulin']]]).reshape(1,-1)
        new_m = np.array([data_init.loc[ind,'bg_postprandial']]).reshape(-1,1)
        bgl_model.update_lts(new_x, new_m)

    return bgl_model

In [ ]:
def generate_data(patient):
    data = pd.DataFrame(np.zeros((num_rec,len(cvars))),columns=cvars)
    
    data.loc[:,'patient'] = patient
    np.random.seed(2)
    data.loc[:,'meal'] = np.random.randint(60, size=num_rec) + 20 #(20,80)
    np.random.seed(3)
    data.loc[:,'bg_fasting'] = np.random.randint(50, size=num_rec) + 100  # [100,150]

    for i in range(rec_per_event):
        l_ind = int(i*num_event)
        u_ind = int((i+1)*num_event - 1)
        data.loc[l_ind:u_ind,'meal'] = data.loc[0:num_event-1,'meal'].to_numpy()
        data.loc[l_ind:u_ind,'bg_fasting'] = data.loc[0:num_event-1,'bg_fasting'].to_numpy()

    return (data, data.loc[0:num_event-1,:])

In [ ]:
def get_init_data(cur_patient, N_init, postbg_target, cr, cf, data, bgl_vals, ins_spacing):
    cvars_init = ['patient', 'meal', 'insulin', 'bg_fasting', 'bg_postprandial']
    data_init = pd.DataFrame(np.zeros((N_init, len(cvars_init))),columns=cvars_init)
    c_list = []
    i_list = []
    bg_list = []
    for i in range(N_init):
        data_init.loc[i,'patient'] = cur_patient
        meal_ind = i+1
        data_init.loc[i,'meal'] = data['meal'][meal_ind] 
        data_init.loc[i,'bg_fasting'] = data['bg_fasting'][meal_ind] 
        insulin = np.minimum(std_bolus_calc(cho=data_init.loc[i,'meal'], gm=data_init.loc[i,'bg_fasting'], gt=postbg_target, cr=cr, cf=cf), ins_ceiling - 0.1)
        data_init.loc[i,'insulin'] = insulin
        data_init.loc[i,'bg_postprandial'] = bgl_vals[meal_ind, int(insulin/ins_spacing)] + np.random.normal(loc=0, scale=sigma_noise)

        c_list.append({'meal': data_init.loc[i, 'meal'], 'bg_fasting': data_init.loc[i, 'bg_fasting']})
        i_list.append(data_init.loc[i,'insulin'])
        bg_list.append(data_init.loc[i,'bg_postprandial'])

    return (data_init, c_list, i_list, bg_list)

In [ ]:
data_dict = {}
data_nomeal_dict = {}
cr_vals = {pt: df.loc[df['Name']==pt]['CR'].values[0] - cr_correct[pt] for pt in all_patients}
cf_vals = {pt: df.loc[df['Name']==pt]['CF'].values[0] - cf_correct[pt] for pt in all_patients}

# generate experiment data
data_exp_all, data_pre_all = generate_data(patient='dummy_patient')

for p_ind, cur_patient in enumerate(all_patients):

    bgl_vals = np.load('./../calc_res/{}_calc_res.npy'.format(cur_patient))

    if cur_patient in child_patients:
        ins_ceiling = 10
        aux_ins = 20
    elif cur_patient in normal_patients:
        ins_ceiling = 20
        aux_ins = 10
    elif cur_patient in hard_patients:
        ins_ceiling = 35
        aux_ins = 10
    else:
        ins_ceiling = 80
        aux_ins = 5

    ins_spacing = 1/aux_ins

    pre_cr = df.loc[df['Name']==cur_patient]['CR'].values[0]
    pre_cf = df.loc[df['Name']==cur_patient]['CF'].values[0]

    cr = cr_vals[cur_patient]
    cf = cf_vals[cur_patient]

    # generate experiment data
    data_exp = data_exp_all.copy()
    data_pre = data_pre_all.copy()
    data_exp['patient'] = cur_patient
    data_pre['patient'] = cur_patient
    
    nomeal_bgs = [70, 75, 80, 85, 90]
    data_nomeal = pd.DataFrame(np.zeros((len(nomeal_bgs),len(cvars_nomeal))), columns=cvars_nomeal)

    ## GENERATE INITIAL DATA
    N_init = 1
    data_init, cons, ins, bgs = get_init_data(cur_patient=cur_patient, N_init=N_init, postbg_target=postbg_target, cr=pre_cr, cf=pre_cf,
                                              data=data_pre, bgl_vals=bgl_vals, ins_spacing=ins_spacing)
    
    bgl_model_salelts = create_model(data_init, ins_ceiling=ins_ceiling, postbg_target=postbg_target, bg_omin=70, bg_omax=180)
    bgl_model_lelts = create_model(data_init, ins_ceiling=ins_ceiling, postbg_target=postbg_target, bg_omin=-1e4, bg_omax=1e4)

    ins_calc_tc_dict = {}
    bgl_calc_tc_dict = {}

    ## SEQUENTIALLY LEARN
    for i in range(num_rec):
        meal_ind = i%num_event + 1
        meal_rec_num = i//num_event + 1
        #print('\nPatient: {}, Meal Event: {}, Rec. Num.: {}'.format(cur_patient, meal_ind, meal_rec_num))

        # context
        context={'meal':data_exp['meal'][i], 
                 'bg_fasting':data_exp['bg_fasting'][i]}

        if meal_rec_num == 1:
            ins_calc_tc = std_bolus_calc(cho=context['meal'], gm=context['bg_fasting'], gt=postbg_target, cr=cr, cf=cf)
            ins_calc_tc = min(ins_calc_tc, ins_ceiling)
            bgl_calc_tc = bgl_vals[meal_ind - 1, int(ins_calc_tc/ins_spacing)]
            ins_calc_tc_dict[meal_ind] = ins_calc_tc
            bgl_calc_tc_dict[meal_ind] = bgl_calc_tc
        else:
            ins_calc_tc = ins_calc_tc_dict[meal_ind]
            bgl_calc_tc = bgl_calc_tc_dict[meal_ind]

        ins_salelts = bgl_model_salelts.recommend(context, strategy='salelts', ins_calc=ins_calc_tc, ins_ceiling=ins_ceiling, aux_ins=aux_ins)
        ins_lelts = bgl_model_lelts.recommend(context, strategy='lelts', ins_calc=ins_calc_tc, ins_ceiling=ins_ceiling, aux_ins=aux_ins)

        ins_salelts = min(ins_ceiling, ins_salelts)
        ins_lelts = min(ins_ceiling, ins_lelts)

        data_exp.loc[i,'ins_salelts'] = ins_salelts
        data_exp.loc[i,'ins_lelts'] = ins_lelts
        data_exp.loc[i,'ins_calc_tc'] = ins_calc_tc 
        
        bgl_salelts = bgl_vals[meal_ind - 1, int(ins_salelts/ins_spacing)]
        bgl_lelts = bgl_vals[meal_ind - 1, int(ins_lelts/ins_spacing)]
         
        data_exp.loc[i,'SALE-LTS-N'] = bgl_salelts
        data_exp.loc[i,'LE-LTS-N'] = bgl_lelts
        data_exp.loc[i,'Tuned Calc.-N'] = bgl_calc_tc

        data_exp.loc[i,'SALE-LTS'] = bgl_salelts + np.random.normal(loc=0, scale=sigma_noise)
        data_exp.loc[i,'LE-LTS'] = bgl_lelts + np.random.normal(loc=0, scale=sigma_noise)
        data_exp.loc[i,'Tuned Calc.'] = bgl_calc_tc + np.random.normal(loc=0, scale=sigma_noise)

        bgl_model_salelts.update_lts(np.array([[context['meal']],[context['bg_fasting']], [ins_salelts]]).reshape(1,3), np.array([data_exp.loc[i,'SALE-LTS']]).reshape(-1,1))
        bgl_model_lelts.update_lts(np.array([[context['meal']],[context['bg_fasting']], [ins_lelts]]).reshape(1,3), np.array([data_exp.loc[i,'LE-LTS']]).reshape(-1,1))

    data_exp.to_csv('./ppbg/{}_mme.csv'.format(cur_patient))
    data_dict[cur_patient] = data_exp

    for i in range(len(data_nomeal)):
        context={'meal':0, 
                 'bg_fasting':nomeal_bgs[i]}
        
        ins_calc_tc = std_bolus_calc(cho=context['meal'], gm=context['bg_fasting'], gt=postbg_target, cr=cr, cf=cf)
        ins_calc_tc = min(ins_calc_tc, ins_ceiling)
        ins_salelts = bgl_model_salelts.recommend(context, strategy='salelts', ins_calc=ins_calc_tc, ins_ceiling=ins_ceiling, aux_ins=aux_ins)
        ins_salelts = min(ins_ceiling, ins_salelts)

        data_nomeal.loc[i,'patient'] = cur_patient
        data_nomeal.loc[i,'meal'] = context['meal']
        data_nomeal.loc[i,'bg_fasting'] = context['bg_fasting']
        data_nomeal.loc[i,'ins_salelts'] = ins_salelts

        data_nomeal.to_csv('./ppbg/{}_nomeal.csv'.format(cur_patient))
        data_nomeal_dict[cur_patient] = data_nomeal

data = pd.concat(list(data_dict.values()), ignore_index=True)
data.to_csv('./ppbg/mme_ppbg.csv', index=False)

data_nomeal_all = pd.concat(list(data_nomeal_dict.values()), ignore_index=True)
data_nomeal_all.to_csv('./ppbg/nomeal_all_ppbg.csv', index=False)